In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

base_url = 'https://easycep.com/kategori/cep-telefonu-1?sortingCriteria=2&page='

# Titles corresponding to the information
titles = [
    'Yüz Tanıma', 
    'Parmak İzi Okuyucu', 
    'RAM', 
    'Hızlı Şarj', 
    'Ön Kamera Çözünürlüğü', 
    'Arka Kamera Çözünürlüğü', 
    'Ekran Boyutu', 
    'Hat Sayısı'
]

# List to store all the data for transering to excel
all_data=[]

# Iterate over pages from 1 to 11
for page in range(1, 12):
    url = base_url + str(page)
    response = requests.get(url)
    if response.status_code == 200:
        # Parsing HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the first part of the desired HTML
        f_fragment = soup.find('div', class_='row g-3')
        
        # Control if the row is found
        if f_fragment: 
            #Find the second fragment from the f_fragment
            s_fragment = f_fragment.find_all('div', class_='product__typeOne')
            
            if s_fragment:
                #Extract a and href to get the absolute url
                for i in s_fragment:
                    t_fragment = i.find('a')
                    if t_fragment:
                        href = t_fragment.get('href')
                        # Updated URL
                        absolute_url = urljoin(url, href)
                        
                        # New request to the desired site
                        linked_page_request = requests.get(absolute_url)
                        if linked_page_request.status_code == 200: #Check if the site is found
                            linked_soup = BeautifulSoup(linked_page_request.content, 'html.parser')
                            #To get the technical information table
                            name = linked_soup.find('h2', class_='productDetail__tabList--title').text.strip()
                            print(f'Isim: {name}')
                            table = linked_soup.find_all('div', class_='productDetail__tabList--itemText')
                            #Store the data 
                            data= {'Isim': name}
                            
                            # Check if we have the expected number of items
                            if len(table) == len(titles):
                                #Iterate titles over items of the table
                                for each_title, item in zip(titles, table):  
                                    print(f'{each_title}: {item.text.strip()}')
                                    #Store the data into my list
                                    data[each_title]= item.text.strip()
                            else:
                                #This part is to control the loop, if the numbers are not same
                                print(f'Expected {len(titles)} items, but found {len(table)} items.')
                                for each_title, item in zip(titles, table):
                                    print(f'{each_title}: {item.text.strip()}')
                                    #Store the data
                                    data[each_title]= item.text.strip()
                                for title in titles[len(table):]:
                                    print(f'{title}: Data not found')
                                    
                            
                            # Append the data to the all_data list
                            #all_data.append(data)
                            
                        else:
                            print('Third stage of HTML fragment not found')
            else:
                print('Second stage of HTML fragment not found')
        else: 
            print('First stage of HTML fragment not found')
    else:
        print(f'Failed to retrieve the webpage. Status code: {response.status_code}')

        
# Convert the all_data list
#df = pd.DataFrame(all_data, columns=['Isim'] + titles)

# Save the Data to an Excel file
#df.to_excel('output.xlsx', index=False)
